In [49]:
import itertools
import time
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from statsmodels.regression.linear_model import OLS as OLS
%matplotlib inline
plt.style.use('ggplot')

In [50]:
data_0 = pd.read_csv('final2.csv')
data_0=data_0.drop(['BIRTH','SCREEN','DEPLOY','OTHERDRUGS','METHADONE','ANYOPIATEDAYS','HALLUC','SEX', 'HISPANIC', 'RACE', 'VET', 'ACTIVE','DAYSCOCAINE','METHDAYS','AGE' ],axis=1)
data_0=data_0.dropna()
#data_0.columns.str.strip()
data_0.columns
df=data_0.copy()
df=df.dropna()
df=df.sample(frac=1)
df.columns

Index(['DAST', 'AUDIT', 'COSCREEN', 'BI', 'BT', 'RT', 'ANYALC', 'BINGEDAYS',
       'DRUGDAYS', 'ALCDRUGS', 'MARYJDAYS', 'INJECT', 'TOBMONTH'],
      dtype='object')

In [51]:
df['DAST'] = df['DAST'].astype(float).astype(int)
# df['SEX'] = df['SEX'].astype(float).astype(int)
# df['HISPANIC'] = df['HISPANIC'].astype(float).astype(int)
# df['RACE'] = df['RACE'].astype(float).astype(int)
# df['VET'] = df['VET'].astype(float).astype(int)
# df['ACTIVE'] = df['ACTIVE'].astype(float).astype(int)
# df['DEPLOY'] = df['DEPLOY'].astype(float).astype(int)
df['AUDIT'] = df['AUDIT'].astype(float).astype(int)
df['COSCREEN'] = df['COSCREEN'].astype(float).astype(int)
df['RT'] = df['RT'].astype(float).astype(int)
df['BI'] = df['BI'].astype(float).astype(int)
df['BT'] = df['BT'].astype(float).astype(int)
df['INJECT'] = df['INJECT'].astype(float).astype(int)
df['TOBMONTH'] = df['TOBMONTH'].astype(float).astype(int)
###############################################################
df['ANYALC'] = df['ANYALC'].astype(float).astype(int)
df['BINGEDAYS'] = df['BINGEDAYS'].astype(float).astype(int)
df['DRUGDAYS'] = df['DRUGDAYS'].astype(float).astype(int)
df['ALCDRUGS'] = df['ALCDRUGS'].astype(float).astype(int)
# df['DAYSCOCAINE'] = df['DAYSCOCAINE'].astype(float).astype(int)
df['MARYJDAYS'] = df['MARYJDAYS'].astype(float).astype(int)
# df['METHDAYS'] = df['METHDAYS'].astype(float).astype(int)
# df['AGE'] = df['AGE'].astype(float).astype(int)
df

,DAST,AUDIT,COSCREEN,BI,BT,RT,ANYALC,BINGEDAYS,DRUGDAYS,ALCDRUGS,MARYJDAYS,INJECT,TOBMONTH
6085,1,0,0,1,0,0,0,0,1,0,1,0,1
1336,0,0,0,0,0,0,0,0,0,0,0,0,0
3469,0,0,0,0,0,0,0,0,0,0,0,0,0
3651,0,0,0,0,0,0,0,0,0,0,0,0,0
5629,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,0,0,0,0,0,0,0,0,0,0,0,0,0
2897,0,0,0,0,0,0,0,0,0,0,0,0,1
2231,0,0,0,0,0,0,0,0,0,0,0,0,0
861,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
#'SEX', 'HISPANIC', 'RACE', 'VET', 'ACTIVE', 'DEPLOY', 'AUDIT',
 #      'COSCREEN', 'BI', 'BT', 'RT', 'ANYALC', 
df = pd.get_dummies(df, columns = [  ],drop_first = True)
df.head(3)

,DAST,AUDIT,COSCREEN,BI,BT,RT,ANYALC,BINGEDAYS,DRUGDAYS,ALCDRUGS,MARYJDAYS,INJECT,TOBMONTH
6085,1,0,0,1,0,0,0,0,1,0,1,0,1
1336,0,0,0,0,0,0,0,0,0,0,0,0,0
3469,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import statsmodels
def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    X = sm.add_constant(X)
    model_k = LogisticRegression(solver='newton-cg', multi_class='multinomial').fit(X,Y)
    y_pred=model_k.predict(X)
    RSS = mean_squared_error(Y,y_pred)* len(Y)

    R_squared = model_k.score(X,Y)
    return RSS, R_squared

In [54]:
#Importing tqdm for the progress bar
from tqdm import tnrange, tqdm_notebook, trange
#Initialization variables
Y = df.DAST
X = df.drop(columns = 'DAST', axis = 1)
print(X.columns)
k = 12
RSS_list, R_squared_list, feature_list = [],[], []
numb_features = []

#Looping over k = 1 to k = 11 features in X
for k in trange(1,len(X.columns) + 1, desc = 'Loop...'):

    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(X.columns,k):
        tmp_result = fit_linear_reg(X[list(combo)],Y)   #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
df = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 'R_squared':R_squared_list,'features':feature_list})


Index(['AUDIT', 'COSCREEN', 'BI', 'BT', 'RT', 'ANYALC', 'BINGEDAYS',
       'DRUGDAYS', 'ALCDRUGS', 'MARYJDAYS', 'INJECT', 'TOBMONTH'],
      dtype='object')


Loop...:   0%|                                                                                                                                                                              | 0/12 [00:00<?, ?it/s]/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

...

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

In [57]:
df_min = df[df.groupby('numb_features')[''].transform(min) == df['RSS']]
df_max = df[df.groupby('numb_features')['R_squared'].transform(max) == df['R_squared']]
display(df_min.head(3))
display(df_max.head(3))

KeyError: 'Column not found: '

In [56]:
df['min_RSS'] = df.groupby('numb_features')['RSS'].transform(min)
df['max_R_squared'] = df.groupby('numb_features')['R_squared'].transform(max)
df.head()


,numb_features,RSS,R_squared,features,min_RSS,max_R_squared
0,1,582.0,0.933625,"(AUDIT,)",331.0,0.972399
1,1,525.0,0.936692,"(COSCREEN,)",331.0,0.972399
2,1,474.0,0.966922,"(BI,)",331.0,0.972399
3,1,561.0,0.938007,"(BT,)",331.0,0.972399
4,1,372.0,0.937568,"(RT,)",331.0,0.972399


In [ ]:
fig = plt.figure(figsize = (16,6))
ax = fig.add_subplot(1, 2, 1)

ax.scatter(df.numb_features,df.RSS, alpha = .2, color = 'darkblue' )
ax.set_xlabel('# Features')
ax.set_ylabel('RSS')
ax.set_title('RSS - Best subset selection')
ax.plot(df.numb_features,df.min_RSS,color = 'r', label = 'Best subset')
ax.legend()

ax = fig.add_subplot(1, 2, 2)
ax.scatter(df.numb_features,df.R_squared, alpha = .2, color = 'darkblue' )
ax.plot(df.numb_features,df.max_R_squared,color = 'r', label = 'Best subset')
ax.set_xlabel('# Features')
ax.set_ylabel('R squared')
ax.set_title('R_squared - Best subset selection')
ax.legend()

plt.show()

In [58]:
#Initialization variables
# Y = df.DAST
# X = df.drop(columns = 'DAST', axis = 1)
# k = 12

remaining_features = list(X.columns.values)
features = []
RSS_list, R_squared_list = [np.inf], [np.inf] #Due to 1 indexing of the loop...
features_list = dict()

for i in range(1,k+1):
    best_RSS = np.inf
    
    for combo in itertools.combinations(remaining_features,1):

            RSS = fit_linear_reg(X[list(combo) + features],Y)   #Store temp result 

            if RSS[0] < best_RSS:
                best_RSS = RSS[0]
                best_R_squared = RSS[1] 
                best_feature = combo[0]

    #Updating variables for next loop
    features.append(best_feature)
    remaining_features.remove(best_feature)
    
    #Saving values for plotting
    RSS_list.append(best_RSS)
    R_squared_list.append(best_R_squared)
    features_list[i] = features.copy()

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/sameerahtalafha/anaconda3/envs/py3.8/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = p

In [59]:
print('Forward stepwise subset selection')
print('Number of features |', 'Features |', 'RSS')
display([(i,features_list[i], round(RSS_list[i])) for i in range(1,5)])

Forward stepwise subset selection
Number of features | Features | RSS


[(1, ['DRUGDAYS'], 331),
 (2, ['DRUGDAYS', 'RT'], 277),
 (3, ['DRUGDAYS', 'RT', 'BINGEDAYS'], 123),
 (4, ['DRUGDAYS', 'RT', 'BINGEDAYS', 'BT'], 118)]

In [60]:
df1 = pd.concat([pd.DataFrame({'features':features_list}),pd.DataFrame({'RSS':RSS_list, 'R_squared': R_squared_list})], axis=1, join='inner')
df1['numb_features'] = df1.index

In [61]:
#Initializing useful variables
m = len(Y)
p = 11
hat_sigma_squared = (1/(m - p -1)) * min(df1['RSS'])

#Computing
df1['C_p'] = (1/m) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
df1['AIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
df1['BIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] +  np.log(m) * df1['numb_features'] * hat_sigma_squared )
df1['R_squared_adj'] = 1 - ( (1 - df1['R_squared'])*(m-1)/(m-df1['numb_features'] -1))
df1


,features,RSS,R_squared,numb_features,C_p,AIC,BIC,R_squared_adj
1,[DRUGDAYS],331.0,0.972399,1,0.072514,5.325114,5.326522,0.972393
2,"[DRUGDAYS, RT]",277.0,0.973932,2,0.060691,4.456874,4.459689,0.973921
3,"[DRUGDAYS, RT, BINGEDAYS]",123.0,0.976342,3,0.026962,1.979970,1.984193,0.976326
4,"[DRUGDAYS, RT, BINGEDAYS, BT]",118.0,0.978532,4,0.025873,1.899975,1.905606,0.978513
5,"[DRUGDAYS, RT, BINGEDAYS, BT, BI]",108.0,0.978970,5,0.023688,1.739547,1.746586,0.978947
6,"[DRUGDAYS, RT, BINGEDAYS, BT, BI, AUDIT]",73.0,0.992990,6,0.016027,1.176953,1.185399,0.992981
7,"[DRUGDAYS, RT, BINGEDAYS, BT, BI, AUDIT, ANYALC]",68.0,0.993428,7,0.014938,1.096958,1.106812,0.993418
8,"[DRUGDAYS, RT, BINGEDAYS, BT, BI, AUDIT, ANYAL...",64.0,0.993647,8,0.014067,1.033050,1.044311,0.993636
9,"[DRUGDAYS, RT, BINGEDAYS, BT, BI, AUDIT, ANYAL...",63.0,0.993866,9,0.013854,1.017401,1.030070,0.993854
10,"[DRUGDAYS, RT, BINGEDAYS, BT, BI, AUDIT, ANYAL...",62.0,0.994085,10,0.013641,1.001752,1.015830,0.994072


In [62]:
df1.iloc[8,0]

['DRUGDAYS',
 'RT',
 'BINGEDAYS',
 'BT',
 'BI',
 'AUDIT',
 'ANYALC',
 'COSCREEN',
 'MARYJDAYS']